# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from generative_social_choice.utils.helper_functions import get_base_dir_path

In [3]:
from generative_social_choice.slates.voting_algorithms import (
    SequentialPhragmenMinimax,
    GreedyTotalUtilityMaximization,
    ExactTotalUtilityMaximization,
    LPTotalUtilityMaximization,
    VotingAlgorithm,
    GeometricTransformation,
)
from generative_social_choice.slates.voting_algorithm_axioms import (
    IndividualParetoAxiom,
    HappiestParetoAxiom,
    CoverageAxiom,
    MinimumAndTotalUtilityParetoAxiom,
    VotingAlgorithmAxiom,
    NonRadicalMinUtilityAxiom,
    NonRadicalTotalUtilityAxiom,
)
from generative_social_choice.test.utilities_for_testing import rated_vote_cases

# Load Data

In [4]:
voting_algorithm_evals_dir = get_base_dir_path() / "data" / "voting_algorithm_evals"
latest = False
if latest:
    file = sorted(voting_algorithm_evals_dir.glob("*.csv"))[-1]
else:
    file = voting_algorithm_evals_dir / "2025-11-07-234841.csv"
file

df = pd.read_csv(file, index_col=0, header=[0, 1])

In [5]:
file

WindowsPath('G:/Other computers/My Computer/NTFS/dev/generative_social_choice/generative_social_choice/data/voting_algorithm_evals/2025-11-07-234841.csv')

In [6]:
# Select subset of rows
# df = df.iloc[-8:]
df

vote                                                       Simple 1  \
subtest                                            Maximum Coverage   
GreedyTotalUtilityMaximization(utility_transfor...              1.0   
ExactTotalUtilityMaximization(utility_transform...              1.0   
LPTotalUtilityMaximization(utility_transform=None)              1.0   
ExactTotalUtilityMaximization(utility_transform...              1.0   
ReweightedRangeVoting(k=1.0)                                    1.0   
ReweightedRangeVoting(k=0.5)                                    1.0   

vote                                                                                       \
subtest                                            m-th Happiest Person Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                                    1.0   
ExactTotalUtilityMaximization(utility_transform...                                    1.0   
LPTotalUtilityMaximization(utility_transform=None)                                    1.0   
ExactTotalUtilityMaximization(utility_transform...                                    1.0   
ReweightedRangeVoting(k=1.0)                                                          1.0   
ReweightedRangeVoting(k=0.5)                                                          1.0   

vote                                                                             \
subtest                                            Individual Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                          1.0   
ExactTotalUtilityMaximization(utility_transform...                          1.0   
LPTotalUtilityMaximization(utility_transform=None)                          1.0   
ExactTotalUtilityMaximization(utility_transform...                          1.0   
ReweightedRangeVoting(k=1.0)                                                1.0   
ReweightedRangeVoting(k=0.5)                                                1.0   

vote                                                                                                    \
subtest                                            Minimum Utility and Total Utility Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                                                1.0    
ExactTotalUtilityMaximization(utility_transform...                                                1.0    
LPTotalUtilityMaximization(utility_transform=None)                                                1.0    
ExactTotalUtilityMaximization(utility_transform...                                                1.0    
ReweightedRangeVoting(k=1.0)                                                                      1.0    
ReweightedRangeVoting(k=0.5)                                                                      1.0    

vote                                                                                            \
subtest                                            Non-radical Total Utility Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                                         1.0   
ExactTotalUtilityMaximization(utility_transform...                                         1.0   
LPTotalUtilityMaximization(utility_transform=None)                                         1.0   
ExactTotalUtilityMaximization(utility_transform...                                         1.0   
ReweightedRangeVoting(k=1.0)                                                               1.0   
ReweightedRangeVoting(k=0.5)                                                               1.0   

vote                                                       Simple 2  \
subtest                                            Maximum Coverage   
GreedyTotalUtilityMaximization(utility_transfor...              1.0   
ExactTotalUtilityMaximization(utility_transform...              1.0   
LPTotalUtilityMaximization(utility_transform=None)              1.0   
ExactTotalUtilityMaximization(ut

## Overall Performance

In [7]:
df.sum(axis=1)

GreedyTotalUtilityMaximization(utility_transform=None)                             128.083333
ExactTotalUtilityMaximization(utility_transform=None)                              132.916667
LPTotalUtilityMaximization(utility_transform=None)                                 129.875000
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))    134.916667
ReweightedRangeVoting(k=1.0)                                                       124.250000
ReweightedRangeVoting(k=0.5)                                                       125.500000
dtype: float64

In [8]:
pd.DataFrame(np.trunc(df.values + 0/24+1*1e-6), index=df.index, columns=df.columns).sum(axis=1)

GreedyTotalUtilityMaximization(utility_transform=None)                             109.0
ExactTotalUtilityMaximization(utility_transform=None)                              130.0
LPTotalUtilityMaximization(utility_transform=None)                                 126.0
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))    134.0
ReweightedRangeVoting(k=1.0)                                                       105.0
ReweightedRangeVoting(k=0.5)                                                       105.0
dtype: float64

## Differential Test Performance

In [30]:
algs_for_non_uniform_rows = df
# algs_for_non_uniform_rows = df.loc[[ExactTotalUtilityMaximization().name, ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5)).name], :]

non_uniform_columns = algs_for_non_uniform_rows.loc[:, algs_for_non_uniform_rows.nunique() > 1]
non_uniform_columns


vote                                                Ex 1.1 modified  \
subtest                                            Maximum Coverage   
GreedyTotalUtilityMaximization(utility_transfor...         0.916667   
ExactTotalUtilityMaximization(utility_transform...         1.000000   
LPTotalUtilityMaximization(utility_transform=None)         0.916667   
ExactTotalUtilityMaximization(utility_transform...         1.000000   
ReweightedRangeVoting(k=1.0)                               0.916667   
ReweightedRangeVoting(k=0.5)                               0.875000   

vote                                                                                                    \
subtest                                            Minimum Utility and Total Utility Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                                           0.916667    
ExactTotalUtilityMaximization(utility_transform...                                           1.000000    
LPTotalUtilityMaximization(utility_transform=None)                                           1.000000    
ExactTotalUtilityMaximization(utility_transform...                                           1.000000    
ReweightedRangeVoting(k=1.0)                                                                 0.791667    
ReweightedRangeVoting(k=0.5)                                                                 0.958333    

vote                                                                                            \
subtest                                            Non-radical Total Utility Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                                    0.333333   
ExactTotalUtilityMaximization(utility_transform...                                    0.500000   
LPTotalUtilityMaximization(utility_transform=None)                                    0.500000   
ExactTotalUtilityMaximization(utility_transform...                                    1.000000   
ReweightedRangeVoting(k=1.0)                                                          0.666667   
ReweightedRangeVoting(k=0.5)                                                          0.583333   

vote                                                         Ex A.1  \
subtest                                            Maximum Coverage   
GreedyTotalUtilityMaximization(utility_transfor...         0.541667   
ExactTotalUtilityMaximization(utility_transform...         1.000000   
LPTotalUtilityMaximization(utility_transform=None)         1.000000   
ExactTotalUtilityMaximization(utility_transform...         1.000000   
ReweightedRangeVoting(k=1.0)                               0.166667   
ReweightedRangeVoting(k=0.5)                               0.583333   

vote                                                                                       \
subtest                                            m-th Happiest Person Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                               0.958333   
ExactTotalUtilityMaximization(utility_transform...                               1.000000   
LPTotalUtilityMaximization(utility_transform=None)                               1.000000   
ExactTotalUtilityMaximization(utility_transform...                               1.000000   
ReweightedRangeVoting(k=1.0)                                                     0.708333   
ReweightedRangeVoting(k=0.5)                                                     0.833333   

vote                                                                                                    \
subtest                                            Minimum Utility and Total Utility Pareto Efficiency   
GreedyTotalUtilityMaximization(utility_transfor...                                           0.333333    
ExactTotalUtilityMaximization(utility_transform...                                           1.000000    
LPTotalUtilityMaximization(utility_transform=None)         

These counts show which test cases and which axioms most frequently show unique behavior across the `VotingAlgorithm`s

In [10]:
# Count the number of columns with each name in both levels of the MultiIndex
level_0_counts = non_uniform_columns.columns.get_level_values(0).value_counts()
level_1_counts = non_uniform_columns.columns.get_level_values(1).value_counts()

print("Counts for level 0:")
print(level_0_counts)
print("\nCounts for level 1:")
print(level_1_counts)



Counts for level 0:
vote
Ex 1.1 modified    1
Ex 3.1             1
Ex 4.1             1
Ex B.3             1
Ex C.2             1
Ex Alg2.1          1
Name: count, dtype: int64

Counts for level 1:
subtest
Non-radical Total Utility Pareto Efficiency            5
Minimum Utility and Total Utility Pareto Efficiency    1
Name: count, dtype: int64


## Axiom Pass Rate

In [11]:
df.groupby(df.columns.get_level_values(1), axis=1).mean()

C:\Users\aaron\AppData\Local\Temp\ipykernel_23264\1810388850.py:1: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df.groupby(df.columns.get_level_values(1), axis=1).mean()


subtest,Individual Pareto Efficiency,Maximum Coverage,Minimum Utility and Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,m-th Happiest Person Pareto Efficiency
GreedyTotalUtilityMaximization(utility_transform=None),1.0,0.927469,0.913580,0.912037,0.990741
ExactTotalUtilityMaximization(utility_transform=None),1.0,1.000000,1.000000,0.922840,1.000000
LPTotalUtilityMaximization(utility_transform=None),1.0,0.959877,0.962963,0.924383,0.962963
ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5)),1.0,1.000000,0.996914,1.000000,1.000000
ReweightedRangeVoting(k=1.0),1.0,0.848765,0.841049,0.945988,0.966049
ReweightedRangeVoting(k=0.5),1.0,0.864198,0.873457,0.933642,0.976852


# Debuggign

In [12]:
# alg = GreedyTotalUtilityMaximization()
# alg = LPTotalUtilityMaximization()
# alg = SequentialPhragmenMinimax()
# alg = ExactTotalUtilityMaximization()
alg = ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))
# alg = GreedyTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))
# alg = LPTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))
# alg = SequentialPhragmenMinimax(load_magnitude_method="marginal_slate", clear_reassigned_loads=False, redistribute_defected_candidate_loads=False)

In [13]:
df1 = df.loc[alg.name,:].unstack()
df1

subtest,Individual Pareto Efficiency,Maximum Coverage,Minimum Utility and Total Utility Pareto Efficiency,Non-radical Total Utility Pareto Efficiency,m-th Happiest Person Pareto Efficiency
vote,,,,,
Ex 1.1,1.0,1.0,1.000000,1.0,1.0
Ex 1.1 modified,1.0,1.0,1.000000,1.0,1.0
Ex 1.2,1.0,1.0,1.000000,1.0,1.0
Ex 1.3,1.0,1.0,1.000000,1.0,1.0
Ex 2.1,1.0,1.0,1.000000,1.0,1.0
Ex 2.2,1.0,1.0,1.000000,1.0,1.0
Ex 3.1,1.0,1.0,0.916667,1.0,1.0
Ex 4.1,1.0,1.0,1.000000,1.0,1.0
Ex 4.2,1.0,1.0,1.000000,1.0,1.0


In [14]:
case = rated_vote_cases["Ex NRU1"]
case.rated_votes

,s1,s2
0,20,20.100
1,20,19.904


In [15]:
# case = rated_vote_cases["Ex 1.1 modified"]
from generative_social_choice.slates.voting_algorithms import RatedVoteCase
from generative_social_choice.slates.voting_utils import (
    CellWiseAugmentation, CandidateWiseAugmentation, VoterAndCellWiseAugmentation)


# case = rated_vote_cases["Ex A.1"]
case = rated_vote_cases["Ex NRU1"]

# case = RatedVoteCase(
#     name="Ex BIG",
#     rated_votes=case.rated_votes,
#     slate_size=case.slate_size,
#     noise_augmentation_methods = {
#                 CellWiseAugmentation(): 40, 
#                 CandidateWiseAugmentation(): 100, 
#                 VoterAndCellWiseAugmentation(): 100
#             }
# )
# case = rated_vote_cases["Ex 4.3"]
# case = rated_vote_cases["Simple 3"]
# axiom = CoverageAxiom()
# axiom = HappiestParetoAxiom()
# axiom = IndividualParetoAxiom()
# axiom = MinimumAndTotalUtilityParetoAxiom()
axiom = NonRadicalTotalUtilityAxiom()
# axiom = NonRadicalMinUtilityAxiom()

case.rated_votes.tail()

,s1,s2
0,20,20.100
1,20,19.904


In [16]:
alg.name

'ExactTotalUtilityMaximization(utility_transform=GeometricTransformation(p=1.5))'

In [17]:
# df.loc[:,df.columns.get_level_values(1) == axiom.name].iloc[:,10:-10]
df.loc[alg.name,df.columns.get_level_values(1) == axiom.name]
# df.columns.get_level_values(1) == type(axiom).__name__
# df.columns.get_level_values(1), axiom.name


vote             subtest                                    
Simple 1         Non-radical Total Utility Pareto Efficiency    1.0
Simple 2         Non-radical Total Utility Pareto Efficiency    1.0
Simple 3         Non-radical Total Utility Pareto Efficiency    1.0
Ex 1.1           Non-radical Total Utility Pareto Efficiency    1.0
Ex 1.1 modified  Non-radical Total Utility Pareto Efficiency    1.0
Ex 1.2           Non-radical Total Utility Pareto Efficiency    1.0
Ex A.1           Non-radical Total Utility Pareto Efficiency    1.0
Ex 1.3           Non-radical Total Utility Pareto Efficiency    1.0
Ex 2.1           Non-radical Total Utility Pareto Efficiency    1.0
Ex 2.2           Non-radical Total Utility Pareto Efficiency    1.0
Ex 3.1           Non-radical Total Utility Pareto Efficiency    1.0
Ex 4.1           Non-radical Total Utility Pareto Efficiency    1.0
Ex 4.2           Non-radical Total Utility Pareto Efficiency    1.0
Ex 4.3           Non-radical Total Utility Pareto Effic

In [18]:
alg.vote(rated_votes=case.rated_votes, slate_size=case.slate_size)[0]

['s2']

In [19]:
# axiom.satisfactory_slates(rated_votes=case.rated_votes, slate_size=case.slate_size)
# case.augmented_cases[0]

In [20]:
num_aug_cases = len(case.augmented_cases)
aug_case_votes = pd.DataFrame(index = range(num_aug_cases), columns=["rated_votes", "assignments", "axiom_slates", "alg_slate", "satisfied"])
for i, aug_case in enumerate(case.augmented_cases):
    # print(aug_case)
    aug_case_votes.at[i, "rated_votes"] = aug_case
    aug_case_votes.at[i, "axiom_slates"] = axiom.satisfactory_slates(rated_votes=aug_case, slate_size=case.slate_size)
    aug_case_votes.at[i, "alg_slate"], aug_case_votes.at[i, "assignments"] = alg.vote(rated_votes=aug_case, slate_size=case.slate_size)
    aug_case_votes.at[i, "satisfied"] = axiom.evaluate_assignment(
        rated_votes=aug_case,
        assignments=aug_case_votes.at[i, "assignments"],
        slate_size=case.slate_size,
    )
    # print(axiom.satisfactory_slates(rated_votes=aug_case, slate_size=case.slate_size))
# aug_case_votes.iloc[:,2:]

In [21]:
failures = aug_case_votes[aug_case_votes["satisfied"] == False]
failures.iloc[:,2:]


,axiom_slates,alg_slate,satisfied
0,{(s1)},[s2],False


In [22]:
f0 = failures.iloc[0]
f0.rated_votes
# (f0.rated_votes - case.rated_votes)*1e6


,s1,s2
0,20,20.100
1,20,19.904


In [23]:
((f0.rated_votes - case.rated_votes)*1e6)

,s1,s2
0,0.0,0.0
1,0.0,0.0


In [24]:
f0.assignments

,candidate_id
0,s2
1,s2


In [25]:
alg.vote(rated_votes=f0.rated_votes, slate_size=case.slate_size)

(['s2'],
   candidate_id
 0           s2
 1           s2)

In [26]:
axiom.evaluate_assignment(
    rated_votes=f0.rated_votes,
    assignments=f0.assignments,
    slate_size=case.slate_size,
)

False

In [27]:
from kiwiutils.kiwilib import leafClasses
[cls.__name__ for cls in leafClasses(VotingAlgorithmAxiom)]

['IndividualParetoAxiom',
 'CoverageAxiom',
 'NonRadicalMinUtilityAxiom',
 'NonRadicalTotalUtilityAxiom',
 'HappiestParetoAxiom',
 'MinimumAndTotalUtilityParetoAxiom']

# Scratch

In [28]:
alg1 = ExactTotalUtilityMaximization()
_, asg = alg.vote(rated_votes=case.rated_votes, slate_size=case.slate_size)
asg

,candidate_id
0,s2
1,s2


In [29]:
axiom.evaluate_assignment(
    rated_votes=case.rated_votes,
    assignments=asg,
    slate_size=case.slate_size,
)

False